In [15]:
import glob
import os
import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt
import math
%matplotlib inline

In [25]:
class ImagePreprocess(object):
    
    @staticmethod
    def image_bw(image_folder_path, 
                 new_image_folder, 
                 file_type='png'):
        """
        Load old RGB image and convert it to black and white.
        """
        directory = '{}/{}'.format(os.path.dirname(image_folder_path), new_image_folder)
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        filelist = glob.glob('{}/*.{}'.format(image_folder_path, file_type))
        
        for file_name in filelist:
            im = Image.open(file_name)
            im_bw = im.convert('1')
            im_bw.save('{}/bw_{}'.format(directory, os.path.basename(file_name)), file_type)
    
    @staticmethod        
    def image_resize(image_folder_path,
                     new_image_folder,
                     canvas_size=300,
                     file_type='png'):
        """
        Place one image on another image.

        Resize the canvas of old_image_path and store the new image in
        new_image_path. Center the image on the new canvas.
        """
        
        directory = '{}/{}'.format(os.path.dirname(image_folder_path), new_image_folder)
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        filelist = glob.glob('{}/*.{}'.format(image_folder_path, file_type))
        
        for file_name in filelist:
            im = Image.open(file_name)
            canvas_size = max(canvas_size, im.size[0], im.size[1])
        
        for file_name in filelist:
            im = Image.open(file_name)
            old_width, old_height = im.size

            # Center the image
            x1 = int(math.floor((canvas_size - old_width) / 2))
            y1 = int(math.floor((canvas_size - old_height) / 2))

            mode = im.mode
            if len(mode) == 1:  # L, 1
                new_background = (0)
            if len(mode) == 3:  # RGB
                new_background = (0, 0, 0)
            if len(mode) == 4:  # RGBA, CMYK
                new_background = (0, 0, 0, 0)

            new_image = Image.new(mode, (canvas_size, canvas_size), new_background)
            new_image.paste(im, (x1, y1, x1 + old_width, y1 + old_height))
            new_image.save('{}/resize_{}'.format(directory, os.path.basename(file_name)), file_type)

In [26]:
t

'/Users/Walkon302/Desktop/MEMA_data/organized'

In [28]:
ImagePreprocess.image_resize(t, 'size_new', canvas_size=50)

In [3]:
class ImageAugementation(object):
    
    @staticmethod
    def image_to_array(image_file_list):
        return np.array([cv2.imread(file_name, 0) for file_name in image_file_list])
    
    @staticmethod
    def image_rotation(image_array, degree=90):
        result = []
        rows, cols = image_array[0].shape
        for image in image_array:         
            rotate_matrix = cv2.getRotationMatrix2D((cols/2, rows/2), degree ,1)
            image_modified = cv2.warpAffine(image, rotate_matrix, (cols, rows))
            result.append(image_modified)
        return np.array(result)
    
    @staticmethod
    def image_flip(image_array):
        result = []
        for image in image_array:
            x_flip = cv2.flip(image, 0)
            y_flip = cv2.flip(image, 1)
            result.append(x_flip)
            result.append(y_flip)
        return np.array(result)
    
    @staticmethod
    def image_augmentation(image_array):
        flip = ImageAugementation.image_flip(image_array)
        rotation = ImageAugementation.image_rotation(image_array)
        return np.concatenate([image_array, flip, rotation])

In [4]:
good_list = glob.glob('/Users/Walkon302/Desktop/MEMA_data/organized/*')
bad_list = glob.glob('/Users/Walkon302/Desktop/MEMA_data/disorganized/*')
good_array = ImageAugementation.image_to_array(good_list)
bad_array = ImageAugementation.image_to_array(bad_list)
good = ImageAugementation.image_augmentation(good_array)
bad = ImageAugementation.image_augmentation(bad_array)